## Úvod

Tato lekce pokryje:
- Co je volání funkce a její použití
- Jak vytvořit volání funkce pomocí OpenAI
- Jak integrovat volání funkce do aplikace

## Cíle učení

Po dokončení této lekce budete vědět, jak a rozumět:

- Účelu používání volání funkcí
- Nastavení volání funkce pomocí služby OpenAI
- Navrhovat efektivní volání funkcí pro použití ve vaší aplikaci


## Pochopení volání funkcí

Pro tuto lekci chceme vytvořit funkci pro náš vzdělávací startup, která uživatelům umožní použít chatbot k nalezení technických kurzů. Doporučíme kurzy, které odpovídají jejich úrovni dovedností, aktuální roli a zájmu o technologii.

K dokončení použijeme kombinaci:
 - `OpenAI` pro vytvoření chatovacího zážitku pro uživatele
 - `Microsoft Learn Catalog API` k pomoci uživatelům najít kurzy na základě požadavku uživatele
 - `Volání funkcí` k převzetí dotazu uživatele a jeho odeslání do funkce pro provedení požadavku na API.

Pro začátek se podívejme, proč bychom chtěli volání funkcí vůbec použít:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # získejte novou odpověď od GPT, kde může vidět odpověď funkce


print(second_response.choices[0].message)


### Proč volání funkcí

Pokud jste dokončili jakoukoli jinou lekci v tomto kurzu, pravděpodobně chápete sílu používání velkých jazykových modelů (LLM). Doufejme, že také vidíte některá jejich omezení.

Volání funkcí je funkce služby OpenAI navržená k řešení následujících výzev:

Nekonzistentní formátování odpovědí:
- Před voláním funkcí byly odpovědi z velkého jazykového modelu nestrukturované a nekonzistentní. Vývojáři museli psát složitý validační kód, aby zvládli každou variantu výstupu.

Omezená integrace s externími daty:
- Před touto funkcí bylo obtížné začlenit data z jiných částí aplikace do kontextu chatu.

Standardizací formátů odpovědí a umožněním bezproblémové integrace s externími daty volání funkcí zjednodušuje vývoj a snižuje potřebu další validační logiky.

Uživatelé nemohli získat odpovědi jako „Jaké je aktuální počasí ve Stockholmu?“. Je to proto, že modely byly omezeny na dobu, kdy byla data trénována.

Podívejme se na níže uvedený příklad, který ilustruje tento problém:

Řekněme, že chceme vytvořit databázi studentských dat, abychom jim mohli navrhnout správný kurz. Níže máme dva popisy studentů, které jsou velmi podobné v datech, která obsahují.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Chceme to poslat do LLM, aby analyzoval data. To může být později použito v naší aplikaci k odeslání na API nebo uložení do databáze.

Vytvořme dva identické prompt, ve kterých LLM instruujeme, jaké informace nás zajímají:


Chceme to poslat LLM, aby analyzoval části, které jsou důležité pro náš produkt. Takže můžeme vytvořit dva identické podněty k instrukci LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Po vytvoření těchto dvou promptů je odešleme do LLM pomocí `openai.ChatCompletion`. Prompt uložíme do proměnné `messages` a přiřadíme roli `user`. Tím simulujeme zprávu od uživatele, která je psána chatbotovi.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Nyní můžeme odeslat oba požadavky do LLM a prozkoumat odpověď, kterou obdržíme.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

I když jsou výzvy stejné a popisy podobné, můžeme získat různé formáty vlastnosti `Grades`.

Pokud spustíte výše uvedenou buňku několikrát, formát může být `3.7` nebo `3.7 GPA`.

Je to proto, že LLM přijímá nestrukturovaná data ve formě psané výzvy a také vrací nestrukturovaná data. Potřebujeme mít strukturovaný formát, abychom věděli, co očekávat při ukládání nebo používání těchto dat.

Použitím volání funkcí můžeme zajistit, že obdržíme zpět strukturovaná data. Při použití volání funkcí LLM ve skutečnosti žádné funkce nevolá ani nespouští. Místo toho vytvoříme strukturu, kterou má LLM dodržovat ve svých odpovědích. Tyto strukturované odpovědi pak používáme k tomu, abychom věděli, kterou funkci spustit v našich aplikacích.


![Diagram toku volání funkce](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.cs.png)


Poté můžeme vzít to, co funkce vrátí, a poslat to zpět do LLM. LLM pak odpoví pomocí přirozeného jazyka, aby odpověděl na dotaz uživatele.


### Případy použití volání funkcí

**Volání externích nástrojů**  
Chatboti jsou skvělí v poskytování odpovědí na otázky uživatelů. Pomocí volání funkcí mohou chatboti využívat zprávy od uživatelů k dokončení určitých úkolů. Například student může požádat chatbota: „Pošli e-mail mému vyučujícímu, že potřebuji více pomoci s tímto předmětem“. To může vyvolat volání funkce `send_email(to: string, body: string)`.

**Vytváření API nebo databázových dotazů**  
Uživatelé mohou najít informace pomocí přirozeného jazyka, který se převede na formátovaný dotaz nebo API požadavek. Příkladem může být učitel, který požádá: „Kteří studenti dokončili poslední úkol“, což může vyvolat volání funkce `get_completed(student_name: string, assignment: int, current_status: string)`.

**Vytváření strukturovaných dat**  
Uživatelé mohou vzít blok textu nebo CSV a použít LLM k extrakci důležitých informací. Například student může převést článek z Wikipedie o mírových dohodách a vytvořit AI flash karty. To lze provést pomocí funkce `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`.


## 2. Vytvoření vašeho prvního volání funkce

Proces vytvoření volání funkce zahrnuje 3 hlavní kroky:
1. Zavolání API Chat Completions s seznamem vašich funkcí a uživatelskou zprávou
2. Přečtení odpovědi modelu pro provedení akce, tj. spuštění funkce nebo API volání
3. Provedení dalšího volání API Chat Completions s odpovědí z vaší funkce, aby bylo možné použít tyto informace k vytvoření odpovědi uživateli.


![Tok volání funkce](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.cs.png)


### Prvky volání funkce

#### Uživatelský vstup

Prvním krokem je vytvořit uživatelskou zprávu. Ta může být dynamicky přiřazena získáním hodnoty z textového vstupu, nebo můžete hodnotu přiřadit zde. Pokud pracujete s API Chat Completions poprvé, musíme definovat `role` a `content` zprávy.

`role` může být buď `system` (vytváření pravidel), `assistant` (model) nebo `user` (konečný uživatel). Pro volání funkce přiřadíme tuto hodnotu jako `user` a uvedeme příklad otázky.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Vytváření funkcí. 

Dále definujeme funkci a parametry této funkce. Použijeme zde pouze jednu funkci nazvanou `search_courses`, ale můžete vytvořit více funkcí.

**Důležité** : Funkce jsou zahrnuty v systémové zprávě pro LLM a budou započítány do množství dostupných tokenů, které máte k dispozici. 


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definice** 

Struktura definice funkce má několik úrovní, z nichž každá má své vlastní vlastnosti. Zde je rozpis vnořené struktury:

**Vlastnosti funkce na nejvyšší úrovni:**

`name` - Název funkce, kterou chceme zavolat. 

`description` - Popis toho, jak funkce funguje. Zde je důležité být konkrétní a jasný. 

`parameters` - Seznam hodnot a formát, který chcete, aby model ve své odpovědi vytvořil. 

**Vlastnosti objektu parametrů:**

`type` - Datový typ objektu parametrů (obvykle "object")

`properties` - Seznam konkrétních hodnot, které model použije ve své odpovědi. 

**Vlastnosti jednotlivých parametrů:**

`name` - Implicitně definováno klíčem vlastnosti (např. "role", "product", "level")

`type` - Datový typ tohoto konkrétního parametru (např. "string", "number", "boolean") 

`description` - Popis konkrétního parametru 

**Volitelné vlastnosti:**

`required` - Pole uvádějící, které parametry jsou vyžadovány pro dokončení volání funkce.


### Volání funkce  
Po definování funkce ji nyní musíme zahrnout do volání API Chat Completion. To provedeme přidáním `functions` do požadavku. V tomto případě `functions=functions`.  

Existuje také možnost nastavit `function_call` na `auto`. To znamená, že necháme LLM rozhodnout, která funkce by měla být volána na základě uživatelské zprávy, místo abychom ji přiřazovali sami.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Nyní se podívejme na odpověď a jak je formátována:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Vidíte, že je volána funkce podle jména a z uživatelské zprávy byl LLM schopen najít data, která odpovídají argumentům funkce.


## 3.Integrace volání funkcí do aplikace. 


Poté, co jsme otestovali formátovanou odpověď z LLM, nyní ji můžeme integrovat do aplikace. 

### Řízení toku 

Pro integraci do naší aplikace proveďme následující kroky: 

Nejprve proveďme volání služeb OpenAI a uložme zprávu do proměnné nazvané `response_message`. 


In [ ]:
response_message = response.choices[0].message

Nyní definujeme funkci, která zavolá Microsoft Learn API pro získání seznamu kurzů:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Jako nejlepší praxi si pak ukážeme, zda model chce zavolat funkci. Poté vytvoříme jednu z dostupných funkcí a přiřadíme ji k volané funkci.  
Následně vezmeme argumenty funkce a namapujeme je na argumenty z LLM.

Nakonec připojíme zprávu o volání funkce a hodnoty, které byly vráceny zprávou `search_courses`. To dává LLM všechny informace, které potřebuje k odpovědi uživateli pomocí přirozeného jazyka.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Nyní pošleme aktualizovanou zprávu do LLM, abychom mohli obdržet odpověď v přirozeném jazyce místo odpovědi ve formátu JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Výzva kódování

Skvělá práce! Pro pokračování ve vašem učení o volání funkcí OpenAI můžete vytvořit: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Více parametrů funkce, které mohou pomoci studentům najít více kurzů. Dostupné parametry API najdete zde:  
 - Vytvořte další volání funkce, které získá více informací od studenta, například jeho rodný jazyk  
 - Vytvořte zpracování chyb, když volání funkce a/nebo volání API nevrátí žádné vhodné kurzy  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Prohlášení o vyloučení odpovědnosti**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože usilujeme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho mateřském jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Nejsme odpovědní za jakékoliv nedorozumění nebo nesprávné výklady vyplývající z použití tohoto překladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
